## Número Grupo:

# INTEGRANTES DEL GRUPO:

### Nombre 1:
### Nombre 2:
### etc.

# Indicaciones

## A. Objetivos de la tarea

El objetivo de esta tarea es desarrollar y aplicar un modelo de machine learning para predecir el número de personas que no asisten a abordar los vuelos de la aerolínea AeroML, basándose en distintos parámetros como ruta, día de la semana, hora del día, entre otros.

## B. Prerrequisitos para desarrollar la tarea

Antes de trabajar en esta tarea deben haber comprendido los contenidos de la unidad de sobre algoritmos de aprendizaje supervisado, especialmente la temática sobre regresión.

## C. Instrucciones para la elaboración de la tarea

REQUISITOS GENERALES

1. **Análisis exploratorio de los datos**<p>
2. **Preparación de los datos**<p>
3. **Extracción de atributos de la fecha**<p>
4. **Selección de variables**<p>
5. **Desarrollo del modelo**<p>
6. **Reflexione sobre el modelo**<p>
7. **Evaluación del modelo**<p>
8. **Reflexión final**

## Descripción de Variables en el Conjunto de Datos de Vuelos

### Variables Principales

1. **`id`**:  
   - Identificador único para cada registro.
   - Útil para indexación y combinación de datos.
  
2. **`fecha`**:  
   - Fecha del vuelo.
   - Importante para el análisis temporal y la identificación de patrones estacionales o tendencias.
  
3. **`numero_vuelo`**:  
   - Identificador único del vuelo.
   - Sirve para rastrear la eficiencia y popularidad de rutas específicas.
  
4. **`origen` y `destino`**:  
   - Aeropuertos de origen y destino.
   - Pueden usarse para analizar la demanda entre diferentes ubicaciones.

5. **`distancia`**:  
   - Distancia entre el origen y el destino.
   - Puede influir en la tarifa y el consumo de combustible.

6. **`capacidad`**:  
   - Número total de asientos en el avión.
   - Útil para calcular la tasa de ocupación y eficiencia del vuelo.

7. **`venta_usd`**:  
   - Ingresos totales del vuelo en dólares estadounidenses.
   - Un KPI clave para la rentabilidad.

8. **`agendados`**:  
   - Número total de reservas para el vuelo.
   - Indica la demanda y permite calcular la tasa de ocupación.

### Variables de Pasajeros

9. **`inasistencia`**:  
   - Número de pasajeros que no se presentaron.
   - Esto puede afectar la rentabilidad y requiere estrategias de overbooking cuidadosas.

10. **`vuelo_denegado`**:  
    - Número de pasajeros que no pudieron abordar debido al exceso de reservas.
    - Importante para evaluar la eficiencia del algoritmo de overbooking.

### Variables de Tarifas

11. **`tarifa_mediabaja`, `tarifa_alta`, `tarifa_mediaalta`, `tarifa_baja`**:  
    - Distribución de los tipos de tarifas adquiridas.
    - Esto puede ayudar a segmentar a los clientes y ajustar las estrategias de precios.

12. **`pax_freqflyer`**:  
    - Número de pasajeros que redimieron millas.
    - Importante para medir el compromiso del cliente.

13. **`agendado_grupal`**:  
    - Número de reservas grupales.
    - Estos suelen tener tarifas más bajas y podrían afectar la rentabilidad.

### Variables de Conexión

14. **`conexion_nacional`, `conexion_internacional`, `sin_conexion`**:  
    - Indican si el vuelo es parte de una conexión nacional, internacional o si los pasajeros no están en una conexión.
    - Esto puede afectar la logística y la planificación.

### Variables Adicionales

15. **`sin_stock`**:  
    - Variable binaria que muestra los días sin capacidad para vender más boletos.
    - Un indicador de alta demanda que podría usarse para ajustar tarifas o frecuencias de vuelo.

16. **`year`, `month`, `day`, `day_of_week`, `hour`**:  
    - Variables temporales que pueden ser útiles para modelar efectos estacionales o patrones diurnos.

### Lo primero es leer el dataframe y comprender sus características y estadística descriptiva básica.

In [1]:
import pandas as pd
import numpy as np

# Ruta al archivo CSV 
ruta_archivo = 'datos_vuelos_AeroML.csv'

# Lectura del archivo CSV en un DataFrame de pandas
df = pd.read_csv(ruta_archivo)

print(df.info())
print(df.head())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18322 entries, 0 to 18321
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      18322 non-null  int64  
 1   fecha                   18322 non-null  object 
 2   numero_vuelo            18322 non-null  int64  
 3   origen                  18322 non-null  object 
 4   destino                 18322 non-null  object 
 5   distancia               18322 non-null  int64  
 6   inasistencia            18322 non-null  int64  
 7   vuelo_denegado          18322 non-null  int64  
 8   tarifa_mediabaja        17315 non-null  float64
 9   tarifa_alta             18322 non-null  int64  
 10  tarifa_mediaalta        18322 non-null  int64  
 11  tarifa_baja             18322 non-null  int64  
 12  pax_freqflyer           18322 non-null  int64  
 13  agendado_grupal         18322 non-null  int64  
 14  sin_stock               18322 non-null

### Según lo observado se corregirán las columnas 'Fecha' y 'hora_salida'. LA primera se transformará a tipo de dato datetime64[ns], mientras a la segunda se extraerá únicamenta la hora y se mantendrá su tipo de dato object. 

In [2]:
# Convertir 'fecha' a tipo datetime
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# Convertir 'hora_salida' a tipo datetime y extraer solo la hora
df['hora_salida'] = pd.to_datetime(df['hora_salida'], errors='coerce').dt.time

# Mostrar las primeras filas para ver el resultado
print(df[['fecha', 'hora_salida']].head())
print(df.info())

       fecha hora_salida
0 2009-02-23    21:15:00
1 2010-01-13    18:15:00
2 2010-10-04    17:26:00
3 2009-12-11    20:20:00
4 2011-11-20    07:47:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18322 entries, 0 to 18321
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      18322 non-null  int64         
 1   fecha                   18322 non-null  datetime64[ns]
 2   numero_vuelo            18322 non-null  int64         
 3   origen                  18322 non-null  object        
 4   destino                 18322 non-null  object        
 5   distancia               18322 non-null  int64         
 6   inasistencia            18322 non-null  int64         
 7   vuelo_denegado          18322 non-null  int64         
 8   tarifa_mediabaja        17315 non-null  float64       
 9   tarifa_alta             18322 non-null  int64         
 10  tarifa_mediaalta

### Ahora, para facilidad de visualización se despliega una tabla estilizada.

In [3]:
df.head().style.set_properties(**{'background-color': '#f0f0f0', 'color': 'black'})

,id,fecha,numero_vuelo,origen,destino,distancia,inasistencia,vuelo_denegado,tarifa_mediabaja,tarifa_alta,tarifa_mediaalta,tarifa_baja,pax_freqflyer,agendado_grupal,sin_stock,conexion_nacional,conexion_internacional,sin_conexion,hora_salida,capacidad,venta_usd,agendados
0,69922,2009-02-23 00:00:00,8942,ANF,SCL,1106,7,0,124.000000,5,1,109,20,0,0,0,0,259.000000,21:15:00,168,8399.700000,259.000000
1,469723,2010-01-13 00:00:00,8941,SCL,ANF,1106,18,0,56.000000,0,1,48,4,0,0,7,6,96.000000,18:15:00,174,8535.700000,109.000000
2,779308,2010-10-04 00:00:00,9128,ANF,SCL,1106,6,0,1.000000,0,0,76,2,0,0,0,1,78.000000,17:26:00,218,3525.400000,79.000000
3,429392,2009-12-11 00:00:00,7941,SCL,ANF,1106,10,0,122.000000,2,4,93,14,0,0,11,1,223.000000,20:20:00,174,10578.400000,235.000000
4,1286557,2011-11-20 00:00:00,9139,SCL,ANF,1106,8,0,1.000000,0,0,185,14,0,1,0,1,199.000000,07:47:00,220,5769.400000,200.000000


## Desarrollo

# 1. Análisis Exploratorio y Preparación de Datos

Realice un análisis exploratorio y una preparación de datos que incluya:

**Comentarios:**
- Comente si existen datos faltantes, si hay valores que no tienen sentido, si hay outliers, etc.
- Describa las decisiones que tomó para la limpieza, imputación y/o transformación de los datos.

### a) La gestión de valores faltantes o nulos (si existieran).

La única columna con valores faltantes es tarifa_mediabaja tiene 17,315 valores no nulos de un total de 18,322.
Eso implica 1,007 valores faltantes (~5.5% del total).

Rellenaremos los valores nulos usando la mediana (median) esta estrategia es más simple considerando porcentaje de nulos no es demasiado alto.

In [4]:
df['tarifa_mediabaja'] = df['tarifa_mediabaja'].fillna(df['tarifa_mediabaja'].median())
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18322 entries, 0 to 18321
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      18322 non-null  int64         
 1   fecha                   18322 non-null  datetime64[ns]
 2   numero_vuelo            18322 non-null  int64         
 3   origen                  18322 non-null  object        
 4   destino                 18322 non-null  object        
 5   distancia               18322 non-null  int64         
 6   inasistencia            18322 non-null  int64         
 7   vuelo_denegado          18322 non-null  int64         
 8   tarifa_mediabaja        18322 non-null  float64       
 9   tarifa_alta             18322 non-null  int64         
 10  tarifa_mediaalta        18322 non-null  int64         
 11  tarifa_baja             18322 non-null  int64         
 12  pax_freqflyer           18322 non-null  int64 

### b) La codificación de variables categóricas (si existieran).

Variables categóricas:
Según .info(), las variables que requieren codificación son:

origen
destino

Estas son variables de tipo object (texto) y representan aeropuertos.

In [5]:
df = pd.get_dummies(df, columns=['origen', 'destino'], drop_first=False)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18322 entries, 0 to 18321
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      18322 non-null  int64         
 1   fecha                   18322 non-null  datetime64[ns]
 2   numero_vuelo            18322 non-null  int64         
 3   distancia               18322 non-null  int64         
 4   inasistencia            18322 non-null  int64         
 5   vuelo_denegado          18322 non-null  int64         
 6   tarifa_mediabaja        18322 non-null  float64       
 7   tarifa_alta             18322 non-null  int64         
 8   tarifa_mediaalta        18322 non-null  int64         
 9   tarifa_baja             18322 non-null  int64         
 10  pax_freqflyer           18322 non-null  int64         
 11  agendado_grupal         18322 non-null  int64         
 12  sin_stock               18322 non-null  int64 

In [6]:
df.head().style.set_properties(**{'background-color': '#f0f0f0', 'color': 'black'})

,id,fecha,numero_vuelo,distancia,inasistencia,vuelo_denegado,tarifa_mediabaja,tarifa_alta,tarifa_mediaalta,tarifa_baja,pax_freqflyer,agendado_grupal,sin_stock,conexion_nacional,conexion_internacional,sin_conexion,hora_salida,capacidad,venta_usd,agendados,origen_ANF,origen_SCL,destino_ANF,destino_SCL
0,69922,2009-02-23 00:00:00,8942,1106,7,0,124.000000,5,1,109,20,0,0,0,0,259.000000,21:15:00,168,8399.700000,259.000000,True,False,False,True
1,469723,2010-01-13 00:00:00,8941,1106,18,0,56.000000,0,1,48,4,0,0,7,6,96.000000,18:15:00,174,8535.700000,109.000000,False,True,True,False
2,779308,2010-10-04 00:00:00,9128,1106,6,0,1.000000,0,0,76,2,0,0,0,1,78.000000,17:26:00,218,3525.400000,79.000000,True,False,False,True
3,429392,2009-12-11 00:00:00,7941,1106,10,0,122.000000,2,4,93,14,0,0,11,1,223.000000,20:20:00,174,10578.400000,235.000000,False,True,True,False
4,1286557,2011-11-20 00:00:00,9139,1106,8,0,1.000000,0,0,185,14,0,1,0,1,199.000000,07:47:00,220,5769.400000,200.000000,False,True,True,False


### c) La normalización o estandarización de los datos.

**ESTANDARIZACIÓN usando StandardScaler**

La estandarización deja todas las variables numéricas con media 0 y desviación estándar 1. También evita que alguna variable con un rango alto (como venta_usd) domine sobre las demás.

**Variables a estandarizar:** Todas las variables numéricas que usará el modelo como predictoras (EXCEPTO la variable objetivo inasistencia y las identificadoras como id o numero_vuelo).

**Variables a estandarizar:**
distancia
vuelo_denegado
tarifa_mediabaja
tarifa_alta
tarifa_mediaalta
tarifa_baja
pax_freqflyer
agendado_grupal
sin_stock
conexion_nacional
conexion_internacional
sin_conexion
capacidad
venta_usd
agendados

In [7]:
from sklearn.preprocessing import StandardScaler

# Selección de columnas numéricas a estandarizar
cols_a_estandarizar = [
    'distancia', 'vuelo_denegado', 'tarifa_mediabaja', 'tarifa_alta',
    'tarifa_mediaalta', 'tarifa_baja', 'pax_freqflyer', 'agendado_grupal',
    'sin_stock', 'conexion_nacional', 'conexion_internacional',
    'sin_conexion', 'capacidad', 'venta_usd', 'agendados'
]

scaler = StandardScaler()
df[cols_a_estandarizar] = scaler.fit_transform(df[cols_a_estandarizar])
df.head().style.set_properties(**{'background-color': '#f0f0f0', 'color': 'black'})

,id,fecha,numero_vuelo,distancia,inasistencia,vuelo_denegado,tarifa_mediabaja,tarifa_alta,tarifa_mediaalta,tarifa_baja,pax_freqflyer,agendado_grupal,sin_stock,conexion_nacional,conexion_internacional,sin_conexion,hora_salida,capacidad,venta_usd,agendados,origen_ANF,origen_SCL,destino_ANF,destino_SCL
0,69922,2009-02-23 00:00:00,8942,0.000000,7,-0.095419,1.556561,0.400102,-0.346699,0.629424,1.176365,-0.297451,-0.280338,-0.568105,-0.676768,1.890233,21:15:00,-0.228229,-0.396707,1.660199,True,False,False,True
1,469723,2010-01-13 00:00:00,8941,0.000000,18,-0.095419,0.135902,-0.476394,-0.346699,-0.874416,-0.940952,-0.297451,-0.280338,-0.182009,0.315820,-0.646149,18:15:00,-0.089775,-0.373922,-0.667289,False,True,True,False
2,779308,2010-10-04 00:00:00,9128,0.000000,6,-0.095419,-1.013160,-0.476394,-0.450121,-0.184129,-1.205617,-0.297451,-0.280338,-0.568105,-0.511337,-0.926240,17:26:00,0.925554,-1.213335,-1.132786,True,False,False,True
3,429392,2009-12-11 00:00:00,7941,0.000000,10,-0.095419,1.514777,-0.125796,-0.036431,0.234974,0.382371,-0.297451,-0.280338,0.038617,-0.511337,1.330051,20:20:00,-0.089775,-0.031694,1.287801,False,True,True,False
4,1286557,2011-11-20 00:00:00,9139,0.000000,8,-0.095419,-1.013160,-0.476394,-0.450121,2.503061,0.382371,-0.297451,3.567120,-0.568105,-0.511337,0.956596,07:47:00,0.971705,-0.837381,0.744721,False,True,True,False


# 2. Selección de Variables y Extracción de Atributos

Elija las variables más relevantes para predecir la inasistencia y realice una extracción de atributos de la fecha. Explique las razones para dejar algunas características fuera.

**Comentarios:**
- Describa las variables que eligió y por qué.
- Explique el proceso de extracción de atributos de la fecha y su relevancia para el modelo.

### **Selección de variables relevantes para predecir inasistencia**

### **Variables incluidas:**

distancia: Puede influir en la decisión de asistir; los vuelos más cortos pueden tener más inasistencias.

vuelo_denegado: El exceso de reservas puede estar relacionado con patrones de inasistencia.

tarifa_mediabaja, tarifa_alta, tarifa_mediaalta, tarifa_baja: El tipo de tarifa comprada puede correlacionar con la propensión a no asistir; tarifas bajas suelen ser menos flexibles y tal vez más propensas a inasistencia.

pax_freqflyer: Los pasajeros frecuentes pueden tener otro comportamiento respecto a la asistencia.

agendado_grupal: Las reservas grupales pueden presentar patrones distintos de inasistencia.

sin_stock: Indica vuelos con muy alta demanda, lo que puede afectar la proporción de no presentados.

conexion_nacional, conexion_internacional, sin_conexion: La conexión puede impactar la probabilidad de no presentarse al vuelo.

capacidad: Puede influir, ya que en vuelos con más capacidad puede haber diferentes políticas de reserva y asistencia.

venta_usd: Representa el ingreso del vuelo; puede correlacionarse indirectamente con la demanda y la inasistencia.

agendados: Número de reservas realizadas, directamente vinculado a la gestión de asientos e inasistencia.

origen_XXX, destino_XXX: Variables dummy del one-hot encoding de aeropuerto, porque la ruta origendestino puede ser relevante para la inasistencia.

Atributos extraídos de la fecha.


### **Variables excluidas y razones:**

id, numero_vuelo: Son identificadores únicos y no contienen información relevante para la predicción.

fecha: No se usa directamente como variable continua, pero de ella se pueden extraer variables temporales de alto valor predictivo.

hora_salida: Es una variable temporal que debe ser procesada a un valor numérico (por ejemplo, minutos desde medianoche o extraer la hora solamente).<-- REVISAR

inasistencia: Es la variable objetivo, no se incluye como predictor.

## Extracción de atributos de fecha

Proceso:
Para aprovechar la información temporal, se extraen los siguientes atributos de la columna fecha:

Por qué es relevante:

Year: Permite capturar tendencias temporales a largo plazo o cambios de política o estacionalidad año a año.

Month: Permite capturar estacionalidad (meses de alta/baja) y patrones de vacaciones.

Day_of_week: Importante porque el comportamiento de reserva y asistencia varía mucho entre días laborales y fines de semana.

In [8]:
df['year'] = df['fecha'].dt.year
df['month'] = df['fecha'].dt.month
df['day'] = df['fecha'].dt.day
df['day_of_week'] = df['fecha'].dt.weekday  # 0 = lunes
df.head().style.set_properties(**{'background-color': '#f0f0f0', 'color': 'black'})

,id,fecha,numero_vuelo,distancia,inasistencia,vuelo_denegado,tarifa_mediabaja,tarifa_alta,tarifa_mediaalta,tarifa_baja,pax_freqflyer,agendado_grupal,sin_stock,conexion_nacional,conexion_internacional,sin_conexion,hora_salida,capacidad,venta_usd,agendados,origen_ANF,origen_SCL,destino_ANF,destino_SCL,year,month,day,day_of_week
0,69922,2009-02-23 00:00:00,8942,0.000000,7,-0.095419,1.556561,0.400102,-0.346699,0.629424,1.176365,-0.297451,-0.280338,-0.568105,-0.676768,1.890233,21:15:00,-0.228229,-0.396707,1.660199,True,False,False,True,2009,2,23,0
1,469723,2010-01-13 00:00:00,8941,0.000000,18,-0.095419,0.135902,-0.476394,-0.346699,-0.874416,-0.940952,-0.297451,-0.280338,-0.182009,0.315820,-0.646149,18:15:00,-0.089775,-0.373922,-0.667289,False,True,True,False,2010,1,13,2
2,779308,2010-10-04 00:00:00,9128,0.000000,6,-0.095419,-1.013160,-0.476394,-0.450121,-0.184129,-1.205617,-0.297451,-0.280338,-0.568105,-0.511337,-0.926240,17:26:00,0.925554,-1.213335,-1.132786,True,False,False,True,2010,10,4,0
3,429392,2009-12-11 00:00:00,7941,0.000000,10,-0.095419,1.514777,-0.125796,-0.036431,0.234974,0.382371,-0.297451,-0.280338,0.038617,-0.511337,1.330051,20:20:00,-0.089775,-0.031694,1.287801,False,True,True,False,2009,12,11,4
4,1286557,2011-11-20 00:00:00,9139,0.000000,8,-0.095419,-1.013160,-0.476394,-0.450121,2.503061,0.382371,-0.297451,3.567120,-0.568105,-0.511337,0.956596,07:47:00,0.971705,-0.837381,0.744721,False,True,True,False,2011,11,20,6


### Extraer la hora en formato numérico:

Esto permite observar si hay más inasistencias en ciertas franjas horarias.

In [9]:
df['hour'] = pd.to_datetime(df['hora_salida'], format='%H:%M:%S').dt.hour
df.head().style.set_properties(**{'background-color': '#f0f0f0', 'color': 'black'})

,id,fecha,numero_vuelo,distancia,inasistencia,vuelo_denegado,tarifa_mediabaja,tarifa_alta,tarifa_mediaalta,tarifa_baja,pax_freqflyer,agendado_grupal,sin_stock,conexion_nacional,conexion_internacional,sin_conexion,hora_salida,capacidad,venta_usd,agendados,origen_ANF,origen_SCL,destino_ANF,destino_SCL,year,month,day,day_of_week,hour
0,69922,2009-02-23 00:00:00,8942,0.000000,7,-0.095419,1.556561,0.400102,-0.346699,0.629424,1.176365,-0.297451,-0.280338,-0.568105,-0.676768,1.890233,21:15:00,-0.228229,-0.396707,1.660199,True,False,False,True,2009,2,23,0,21.000000
1,469723,2010-01-13 00:00:00,8941,0.000000,18,-0.095419,0.135902,-0.476394,-0.346699,-0.874416,-0.940952,-0.297451,-0.280338,-0.182009,0.315820,-0.646149,18:15:00,-0.089775,-0.373922,-0.667289,False,True,True,False,2010,1,13,2,18.000000
2,779308,2010-10-04 00:00:00,9128,0.000000,6,-0.095419,-1.013160,-0.476394,-0.450121,-0.184129,-1.205617,-0.297451,-0.280338,-0.568105,-0.511337,-0.926240,17:26:00,0.925554,-1.213335,-1.132786,True,False,False,True,2010,10,4,0,17.000000
3,429392,2009-12-11 00:00:00,7941,0.000000,10,-0.095419,1.514777,-0.125796,-0.036431,0.234974,0.382371,-0.297451,-0.280338,0.038617,-0.511337,1.330051,20:20:00,-0.089775,-0.031694,1.287801,False,True,True,False,2009,12,11,4,20.000000
4,1286557,2011-11-20 00:00:00,9139,0.000000,8,-0.095419,-1.013160,-0.476394,-0.450121,2.503061,0.382371,-0.297451,3.567120,-0.568105,-0.511337,0.956596,07:47:00,0.971705,-0.837381,0.744721,False,True,True,False,2011,11,20,6,7.000000


##### **Para evitar sesgo y asegurar que todas las variables numéricas estén en la misma escala, estandarizaremos las columnas year, month, day, day_of_week y hour..**

In [10]:
from sklearn.preprocessing import StandardScaler

cols_nuevas = ['year', 'month', 'day', 'day_of_week', 'hour']
df[cols_nuevas] = StandardScaler().fit_transform(df[cols_nuevas])
df.head().style.set_properties(**{'background-color': '#f0f0f0', 'color': 'black'})

,id,fecha,numero_vuelo,distancia,inasistencia,vuelo_denegado,tarifa_mediabaja,tarifa_alta,tarifa_mediaalta,tarifa_baja,pax_freqflyer,agendado_grupal,sin_stock,conexion_nacional,conexion_internacional,sin_conexion,hora_salida,capacidad,venta_usd,agendados,origen_ANF,origen_SCL,destino_ANF,destino_SCL,year,month,day,day_of_week,hour
0,69922,2009-02-23 00:00:00,8942,0.000000,7,-0.095419,1.556561,0.400102,-0.346699,0.629424,1.176365,-0.297451,-0.280338,-0.568105,-0.676768,1.890233,21:15:00,-0.228229,-0.396707,1.660199,True,False,False,True,-1.074552,-1.236137,0.827507,-1.437570,1.266545
1,469723,2010-01-13 00:00:00,8941,0.000000,18,-0.095419,0.135902,-0.476394,-0.346699,-0.874416,-0.940952,-0.297451,-0.280338,-0.182009,0.315820,-0.646149,18:15:00,-0.089775,-0.373922,-0.667289,False,True,True,False,0.133742,-1.520773,-0.310084,-0.388494,0.726809
2,779308,2010-10-04 00:00:00,9128,0.000000,6,-0.095419,-1.013160,-0.476394,-0.450121,-0.184129,-1.205617,-0.297451,-0.280338,-0.568105,-0.511337,-0.926240,17:26:00,0.925554,-1.213335,-1.132786,True,False,False,True,0.133742,1.040952,-1.333915,-1.437570,0.546897
3,429392,2009-12-11 00:00:00,7941,0.000000,10,-0.095419,1.514777,-0.125796,-0.036431,0.234974,0.382371,-0.297451,-0.280338,0.038617,-0.511337,1.330051,20:20:00,-0.089775,-0.031694,1.287801,False,True,True,False,-1.074552,1.610224,-0.537602,0.660582,1.086633
4,1286557,2011-11-20 00:00:00,9139,0.000000,8,-0.095419,-1.013160,-0.476394,-0.450121,2.503061,0.382371,-0.297451,3.567120,-0.568105,-0.511337,0.956596,07:47:00,0.971705,-0.837381,0.744721,False,True,True,False,1.342036,1.325588,0.486230,1.709658,-1.252224


# 3. Desarrollo y Evaluación del Modelo

Desarrolle uno o más modelos de regresión y evalúe el desempeño de los modelos mediante el error absoluto medio (MAE). Interprete correctamente los resultados y su impacto en la precisión del modelo.

**Comentarios:**
- Describa el modelo que eligió y por qué.
- Comente sobre los resultados obtenidos del MAE y su interpretación.
- Elija otra métrica de evaluación, interprete los resultados y compare con el MAE.

### 3.1 Selección de modelos a evaluar

Evaluaremos los siguientes tres modelos:

**1. Regresión Lineal**

Por qué: Proporciona una línea base sencilla y fácil de interpretar. Permite identificar rápidamente relaciones lineales en los datos y establecer un mínimo de referencia.

Modelo: LinearRegression de scikit-learn.

**2. Árbol de Decisión para Regresión**

Por qué: Permite capturar relaciones no lineales y efectos de interacción entre variables, que son habituales en fenómenos complejos como la inasistencia.

Modelo: DecisionTreeRegressor de scikit-learn.

**3. Random Forest para Regresión**

Por qué: Modelo robusto, reduce sobreajuste y ofrece excelente desempeño en problemas tabulares.

Modelo: RandomForestRegressor de scikit-learn.

### Paso 1: Definición de variables predictoras (X) y variable objetivo (y):

**y** = columna inasistencia

**X** = todas las variables seleccionadas para predicción.

### Paso 2: Separamos los datos en entrenamiento y test (80%/20%):

In [11]:
from sklearn.model_selection import train_test_split

X = df.drop(['inasistencia', 'id', 'numero_vuelo', 'fecha', 'hora_salida'], axis=1)
y = df['inasistencia']

for col in X.columns:
    if X[col].isnull().any():
        if X[col].dtype in ['float64', 'int64']:
            X[col] = X[col].fillna(X[col].median())
        elif X[col].dtype == 'bool':
            X[col] = X[col].fillna(False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Paso 3: Entrenamiento y evaluación de cada modelo:

## Regresión Lineal

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred_linreg = linreg.predict(X_test)
mae_linreg = mean_absolute_error(y_test, y_pred_linreg)

## Árbol de Decisión

In [13]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(random_state=42)
dtr.fit(X_train, y_train)
y_pred_dtr = dtr.predict(X_test)
mae_dtr = mean_absolute_error(y_test, y_pred_dtr)

## Random Forest

In [14]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(X_train, y_train)
y_pred_rfr = rfr.predict(X_test)
mae_rfr = mean_absolute_error(y_test, y_pred_rfr)

### Paso 4: Calcular Error cuadrático medio (MSE o RMSE). Esto porque penaliza más los errores grandes y da otra perspectiva sobre la dispersión de los errores.

In [15]:
rmse_linreg = np.sqrt(mean_squared_error(y_test, y_pred_linreg))
rmse_dtr = np.sqrt(mean_squared_error(y_test, y_pred_dtr))
rmse_rfr = np.sqrt(mean_squared_error(y_test, y_pred_rfr))

### Resultados:

In [16]:
resultados = {
    'Modelo': ['Regresión Lineal', 'Árbol de Decisión', 'Random Forest'],
    'MAE': [mae_linreg, mae_dtr, mae_rfr],
    'RMSE': [rmse_linreg, rmse_dtr, rmse_rfr]
}

tabla_resultados = pd.DataFrame(resultados)

tabla_resultados.style.set_properties(**{'background-color': '#f0f0f0', 'color': 'black'})

,Modelo,MAE,RMSE
0,Regresión Lineal,3.701673,5.197735
1,Árbol de Decisión,4.967531,7.020531
2,Random Forest,3.507539,4.849931


## Interpretación de las métricas

MAE (Mean Absolute Error): Indica en promedio cuánto se equivoca el modelo en sus predicciones, en las mismas unidades que la variable objetivo (en este caso, el número promedio de no-shows). Un valor más bajo significa mayor precisión.

RMSE (Root Mean Squared Error): Penaliza errores grandes más que el MAE, ya que eleva al cuadrado las diferencias antes de promediarlas. Es especialmente útil cuando los errores grandes son costosos o indeseables para el negocio.

## Interpretación:

Random Forest obtiene los valores más bajos tanto en MAE como en RMSE, lo que indica que es el modelo que comete menos errores en promedio y también menos errores grandes.
Regresión Lineal está cerca en desempeño, pero se queda ligeramente por detrás del Random Forest.
Árbol de Decisión es el que obtiene el peor desempeño, con errores significativamente mayores en promedio y también mayores errores extremos.

## Recomendación:

### Mejor modelo:

El Random Forest es la mejor opción, ya que obtiene los mejores resultados tanto en MAE como en RMSE. Esto significa que es el modelo más preciso y el que mejor mitiga los errores grandes, lo cual es especialmente valioso cuando estos pueden afectar de manera considerable la experiencia del cliente o la eficiencia operacional.

### Equilibrio entre MAE y RMSE:

Si el principal objetivo de negocio es minimizar el error promedio en la predicción de no-shows (por ejemplo, para mejorar la planificación operativa del vuelo), el MAE es suficiente y Random Forest continúa siendo superior.
Si los errores grandes son muy costosos (por ejemplo, si una sobreestimación grande puede causar pérdidas de ingresos o problemas logísticos graves), entonces el RMSE cobra aún más importancia, y nuevamente Random Forest resulta ser el más adecuado.

### Conclusión

Se recomienda implementar el modelo Random Forest para la predicción de no-shows, ya que logra el mejor equilibrio entre precisión promedio y control de errores extremos. Es un modelo robusto que generalmente maneja mejor la heterogeneidad y las relaciones no lineales presentes en los datos de vuelos y pasajeros.

# 4. Reflexión Final

**El desarrollo del modelo para predecir no-shows de pasajeros en AeroML ha sido un proceso de mucho aprendizaje y desafiante, permitiéndonos profundizar en las distintas etapas del ciclo de vida de un proyecto de machine learning y hemos experimentado de primera mano la importancia de un enfoque metódico y crítico en cada paso de la construcción de modelos predictivos.**

**Durante la etapa de preprocesamiento de datos, nos enfrentamos al reto de manejar datos incompletos y heterogéneos, especialmente en lo relativo a valores faltantes y diferentes tipos de variables (numéricas, categóricas, temporales). Decidir cómo imputar los datos faltantes, normalizar correctamente las variables y convertir las categorías en representaciones numéricas apropiadas (one-hot encoding) fue clave para asegurar que los modelos pudieran aprender patrones significativos sin verse afectados por problemas en la calidad de los datos originales.**

**En la selección y extracción de características, optamos por priorizar aquellas variables que, desde un punto de vista tanto lógico como exploratorio, tenían mayor potencial predictivo. Decidir excluir identificadores no predictivos o información redundante fue fundamental para evitar ruido y complejidad innecesaria en el modelo.**

**Al llegar al desarrollo y evaluación de modelos nos enfrentamos con varias dificultades técnicas, como las diferencias en el manejo de datos entre modelos y librerías, y la gestión de errores derivados de datos atípicos o incompatibilidades. Resolver estos problemas requirió investigar buenas prácticas, experimentar con diferentes enfoques y "debuggear" de manera paciente cada error hasta llegar a una solución adecuada.**

**La evaluación y selección de modelos resultó ser una etapa crítica. Utilizamos métricas complementarias (MAE y RMSE) para asegurarnos de que el modelo seleccionado no solo minimizara el error promedio, sino que también fuera robusto ante errores grandes que pudieran afectar el negocio. Comparando regresión lineal, árboles de decisión y random forest, el aprendizaje es claro: modelos más complejos y robustos como random forest pueden obtener ventajas considerables cuando los datos presentan relaciones no lineales y múltiples interacciones.**

**En conclusión este proceso nos ha reafirmado que la construcción de modelos de machine learning precisos y confiables depende tanto del conocimiento técnico como de una actitud reflexiva y crítica ante cada decisión. La evaluación rigurosa y la selección adecuada del modelo final son esenciales para asegurar no solo el rendimiento, sino también la utilidad real de la solución en un entorno de negocio.**

### Referencias

Géron, A. (2019). Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems (2nd ed.). O’Reilly Media.

Kuhn, M., & Johnson, K. (2013). Applied Predictive Modeling. Springer. https://doi.org/10.1007/978-1-4614-6849-3